## 사전 설정
1. 도커 설치 (https://www.docker.com/products/docker-desktop/)
2. docker composer up -d

## 문제
웹 광고 클릭 예측 모델을 개발하고, MLflow를 사용하여 실험을 관리하세요. 
주어진 train data를 활용하여 모델을 학습하고, test data에 대한 예측을 수행하세요. 
다양한 하이퍼파라미터와 모델을 실험하고, MLflow를 사용하여 각 실험의 성능을 추적하세요.

## 개선 방안
1. 다양한 하이퍼파라미터 조합으로 실험해보세요. 예를 들어, n_estimators, max_depth, min_samples_split 등의 값을 변경해가며 성능 변화를 관찰하세요.
2. 랜덤 포레스트 외에 다른 모델(예: LightGBM, XGBoost)을 사용해보고, MLflow로 실험을 추적하세요.
3. MLflow의 model registry를 사용하여 최고 성능의 모델을 등록하고 관리해보세요.
4. 실험 과정 중 champion model이 아닌 다른 모델들도 MLflow에 기록해보세요. (hint: `mlflow.start_run(nested=True)`)
5. 특성 중요도(feature importance)를 계산하고 MLflow에 기록해보세요.
6. 교차 검증(cross-validation)을 구현해보세요.
7. MLflow의 autolog 기능을 사용해보고, 수동으로 로깅하는 방식과 비교하여 학습해보시오.
8. BentoML을 사용하여 최고 성능의 모델을 서빙할 수 있는 서비스를 만들어보세요. 

## 데이터 정보

**train.csv [파일]** 
- 시간 순으로 나열된 7일 동안의 웹 광고 클릭 로그
- ID: train 데이터 샘플 고유 ID
- Click: 예측 목표인 클릭 여부
- 0: 클릭하지 않음, 1: 클릭
- F01 ~ F39 : 각 클릭 로그와 연관된 Feature
- 개인정보 보호를 위해 상세 정보는 비식별 처리됨

**test.csv [파일]**
- 시간 순으로 나열된 1일 동안의 웹 광고 클릭 로그
- train 데이터의 다음 날에 해당
- Click이 존재하지 않아, 이를 예측하는 것이 목표
- ID: test 데이터 샘플 고유 ID
- F01 ~ F39 : 각 클릭 로그와 연관된 Feature
- 개인정보 보호를 위해 상세 정보는 비식별 처리됨

**sample_submission.csv [파일] - 제출 양식**
- ID: test 데이터 샘플 고유 ID
- Click: 각 샘플에 대해서 예측한 클릭 확률을 기입하여 제출

## 패키지 로드

In [ ]:
!pip install -r requirements.txt -q

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient


## 환경변수 설정

In [2]:
# Minio 서버 설정
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://localhost:9000'  # Minio 서버 주소 및 포트
os.environ['AWS_ACCESS_KEY_ID'] = 'minio'  # Minio 액세스 키
os.environ['AWS_SECRET_ACCESS_KEY'] = 'miniostorage'  # Minio 시크릿 키

# MLflow 추적 서버 URI 설정
os.environ['MLFLOW_TRACKING_URI'] = 'http://localhost:5001'  # MLflow 서버 주소

## 데이터 로드

In [3]:
origin_data = pd.read_csv('data/train.csv', nrows=100000)

In [20]:
origin_data.head(1).to_json()

'{"ID":{"0":"TRAIN_00000000"},"Click":{"0":1},"F01":{"0":"NSLHFNS"},"F02":{"0":"AVKQTCL"},"F03":{"0":"DTZFPRW"},"F04":{"0":114.0},"F05":{"0":"ISVXFVA"},"F06":{"0":1},"F07":{"0":"PQZBVMG"},"F08":{"0":"LPYPUNA"},"F09":{"0":"IZYJZDA"},"F10":{"0":"RANQNXO"},"F11":{"0":66.0},"F12":{"0":"EGWPZEB"},"F13":{"0":"SMRBWMU"},"F14":{"0":4},"F15":{"0":"NGMRRQG"},"F16":{"0":"NLHSWSR"},"F17":{"0":"SXZLOWA"},"F18":{"0":null},"F19":{"0":1.0},"F20":{"0":"LTCDFSX"},"F21":{"0":"SWAZXZY"},"F22":{"0":"SNDDHSM"},"F23":{"0":"IDHAIQQ"},"F24":{"0":5.0},"F25":{"0":"HLADEES"},"F26":{"0":"XAUNDQW"},"F27":{"0":3.0},"F28":{"0":"MAVCFCM"},"F29":{"0":1.0},"F30":{"0":"NZGEZLW"},"F31":{"0":"GTISJWW"},"F32":{"0":380.0},"F33":{"0":2.0},"F34":{"0":"AXQFZWC"},"F35":{"0":"IRUDRFB"},"F36":{"0":null},"F37":{"0":"TFJMLCZ"},"F38":{"0":0.0},"F39":{"0":"AURZYDY"}}'

In [4]:
origin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 41 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      100000 non-null  object 
 1   Click   100000 non-null  int64  
 2   F01     96322 non-null   object 
 3   F02     96322 non-null   object 
 4   F03     59702 non-null   object 
 5   F04     80125 non-null   float64
 6   F05     96322 non-null   object 
 7   F06     100000 non-null  int64  
 8   F07     100000 non-null  object 
 9   F08     100000 non-null  object 
 10  F09     100000 non-null  object 
 11  F10     96322 non-null   object 
 12  F11     90996 non-null   float64
 13  F12     96322 non-null   object 
 14  F13     100000 non-null  object 
 15  F14     100000 non-null  int64  
 16  F15     59702 non-null   object 
 17  F16     100000 non-null  object 
 18  F17     100000 non-null  object 
 19  F18     74106 non-null   float64
 20  F19     87819 non-null   float64
 21  F20     597

In [9]:
train_data = origin_data[:80000]
test_data = origin_data[80000+1:]

In [10]:
# for debug
train_data = origin_data[:800]
test_data = origin_data[800+1:]

## 모델 학습

In [11]:
def preprocess_pipeline(df):
    """
    데이터프레임의 수치형 및 범주형 특성을 자동으로 감지하고,
    각각에 대한 전처리 파이프라인을 생성합니다.

    Args:
    df (DataFrame): 전처리할 데이터프레임

    Return:
    ColumnTransformer: 수치형 및 범주형 특성을 처리하는 전처리 파이프라인
    """
    # 수치형과 범주형 컬럼 자동 감지
    numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = df.select_dtypes(include=['object', 'category']).columns

    # 수치형 특성을 위한 파이프라인
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    # 범주형 특성을 위한 파이프라인
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # ColumnTransformer를 사용하여 전체 전처리 파이프라인 생성
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])

    return preprocessor

In [14]:
# 특성과 타겟 분리
X = train_data.drop(['ID', 'Click'], axis=1)
y = train_data['Click']

# 전처리 파이프라인 생성
preprocessor = preprocess_pipeline(X)

# 학습 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# MLflow 실험 시작
mlflow.set_experiment("mlops-test")

with mlflow.start_run():
    # 하이퍼파라미터 설정
    n_estimators = 100
    max_depth = 10
    min_samples_split = 2

    # 모델 학습
    model = RandomForestClassifier(n_estimators=n_estimators, 
                                   max_depth=max_depth, 
                                   min_samples_split=min_samples_split, 
                                   random_state=42)

    # 파이프라인 생성 (전처리 + 모델)
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    pipeline.fit(X_train, y_train)

    # 예측
    y_pred = pipeline.predict(X_val)
    y_pred_proba = pipeline.predict_proba(X_val)[:, 1]

    # 성능 평가
    accuracy = accuracy_score(y_val, y_pred)
    roc_auc = roc_auc_score(y_val, y_pred_proba)

    # MLflow에 메트릭 기록
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("min_samples_split", min_samples_split)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("roc_auc", roc_auc)

    # 모델 저장
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="model",
        registered_model_name="web-ctr"
    )

    print(f"Accuracy: {accuracy}")
    print(f"ROC AUC: {roc_auc}")

    


Registered model 'web-ctr' already exists. Creating a new version of this model...
2024/07/22 02:38:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: web-ctr, version 4


Accuracy: 0.7625
ROC AUC: 0.6788179465056083


Created version '4' of model 'web-ctr'.


## alias `staging`으로 변경

In [16]:
# Model Registry에서 최신 버전을 가져와 'staging' alias 지정
client = MlflowClient()
latest_version = client.get_latest_versions("web-ctr", stages=["None"])[0]
client.set_registered_model_alias(
    name="web-ctr",
    alias="Staging",
    version=latest_version.version
)


/var/folders/fr/0y5tr38s07gbht7cp2ryt1gr0000gn/T/ipykernel_34637/1935373080.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_version = client.get_latest_versions("web-ctr", stages=["None"])[0]


## bentoml to mlflow

In [17]:
import bentoml

# MLflow 모델 로드
model_name = "web-ctr"
model_alias = "Staging"

prod_model = mlflow.sklearn.load_model(f"models:/{model_name}@{model_alias}")

In [18]:
prod_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['F04', 'F06', 'F11', 'F14', 'F18', 'F19', 'F24', 'F27', 'F29', 'F32',
       'F33', 'F36', 'F38'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['F01', 'F02', 'F03', 'F05', 'F07', 'F08', 'F09', 'F10', 'F12', 'F13',
       'F15', 'F16', 'F17', 'F20', 'F21', 'F22', 'F23', 'F25', 'F26', 'F28',
       'F30', 'F31', 'F34', 'F35', 'F37', 'F39'],
      dtype='object'))])),
                ('model',
                 RandomForestClassifier(max_depth=10, random_state=42))])

In [19]:
# BentoML에 모델 저장
saved_model = bentoml.sklearn.save_model(
    model_name,
    prod_model,
    signatures={
        "predict": {"batchable": True},
        "predict_proba": {"batchable": True}
    }
)
print(f"Model saved: {saved_model}")


Model saved: Model(tag="web-ctr:b44z7bchrcex5lg6")


## bentoml serving serving

1. service.py 작성
2. bentofile.yaml 작성
3. bentoml build
4. bentoml list
5. bentoml serve web-ctr:latest -p 9083                                                                            
6. (optional) bentoml containerize web_ctr:latest
7. (optional) docker run --rm -p 9081:3000 web-ctr:kv3olachq2pv5lg6

In [ ]:
import numpy as np
import pandas as pd
import bentoml
from bentoml.io import NumpyNdarray, PandasDataFrame

web_ctr_runner = bentoml.sklearn.get("web-ctr:latest").to_runner()

svc = bentoml.Service("web-ctr", runners=[web_ctr_runner])

@svc.api(input=PandasDataFrame(), output=NumpyNdarray())
async def predict(df: pd.DataFrame) -> np.ndarray:
    return await web_ctr_runner.predict.async_run(df)

@svc.api(input=PandasDataFrame(), output=NumpyNdarray())
async def predict_proba(df: pd.DataFrame) -> np.ndarray:
    return await web_ctr_runner.predict_proba.async_run(df)


